Resultados esperados:
 - input: 
    - avg_runtime
    - input_complexity
 - output: 
    - p_cores


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json('dataset/freqmine-pthreads_g_10.json')

In [3]:
def calculate_average(data_list, start_index, end_index):
    return np.mean([data['stop_time'] - data['start_time'] for data in data_list[start_index:end_index]])

In [4]:
def extract_avg_runtime(df):
    data_list = df[df['data'].notna()]['data']
    keys = data_list.index

    avgs = []
    BATCH_SIZE = 10

    for i in range(0, len(data_list), BATCH_SIZE):
        avg = calculate_average(data_list, i, i + BATCH_SIZE)
        config = keys[i].split(";")
        avgs.append((config[0], config[1], avg))

    df_data = pd.DataFrame(avgs, columns=['cores', 'input', 'average_runtime'])
    return df_data

In [2]:
def extract_avg_runtime_without_agg(df):
    data_list = df[df['data'].notna()]['data']
    keys = data_list.index

    avgs = []
    BATCH_SIZE = 1

    for i in range(0, len(data_list), BATCH_SIZE):
        avg = calculate_average(data_list, i, i + BATCH_SIZE)
        config = keys[i].split(";")
        avgs.append((config[0], config[1], avg))

    df_data = pd.DataFrame(avgs, columns=['cores', 'input', 'average_runtime'])
    return df_data

In [5]:
df_data = extract_avg_runtime(df)

In [6]:
df_data['input'] = df_data['input'].astype(int)
df_data['cores'] = df_data['cores'].astype(int)

In [40]:
# Calculando Speedup

reference_input = 0
reference_runtime = df_data[(df_data["input"] == reference_input) & (df_data["cores"] == 1)]["average_runtime"].values[0]

df_data['speedup'] = reference_runtime / df_data['average_runtime']

310    0.106112
Name: average_runtime, dtype: float64

In [7]:
df_data.to_csv('dataset/clean_data.csv', index=False)